In [ ]:
import torch


device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

device

In [ ]:
usr = "char"
# Tengo GPU pero no cuda compatible, me rompe cuando intento correr con GPU
if usr == "nico":
    device="cpu"
    num_workers = 4
    
if usr == "char":
    num_workers = 0

In [ ]:
from catdog.dataset import CatDogDataset

In [ ]:
DATA_DIR = '../data/'

In [ ]:
import pandas as pd


train_set_pd = pd.read_csv(DATA_DIR + 'train.csv')
train_set_torch = CatDogDataset(train_set_pd, img_output_size=(150, 150))
print(f"Loaded train set with length {len(train_set_torch)}")

test_set_pd = pd.read_csv(DATA_DIR + 'test.csv')
test_set_torch = CatDogDataset(test_set_pd, img_output_size=(150, 150))
print(f"Loaded test set with length {len(test_set_torch)}")

In [ ]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_set_torch, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_set_torch, batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [ ]:
import numpy as np


seed = 142
torch.manual_seed(seed)
np.random.seed(seed)

#### test image

import numpy as np
from PIL import Image


test_img_path = DATA_DIR + 'images/' + cat_dog_df["file"].iloc[0]
test_img = Image.open(test_img_path).convert("RGB")

In [ ]:
from catdog.models.feedforward import MLPClassifier
from catdog.models.convolutional import ConvolutionalClassifier


# model = MLPClassifier(input_size=500*500*3, hidden_sizes=(50, 10))
model = ConvolutionalClassifier(input_shape=(3,150,150), optimizer_params={"lr": 1e-4})

In [ ]:
d = torch.stack((train_set_torch[0][0], train_set_torch[1][0])).float()

In [ ]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

seed_everything(142, workers=True)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='./lightning_checkpoints/',
    filename='test_model_training',
    mode='min',
    save_top_k=3
)

logger = TensorBoardLogger("tb_logs", name="Conv_classifier")

trainer = Trainer(gpus=0,#int(device=='cuda'), # hacky way to say 0 or 1 
                  max_epochs=1000,
                  logger=logger,
                  callbacks=[EarlyStopping(monitor="val_loss", patience=15, mode='min'),
                             checkpoint_callback])
trainer.fit(model, train_loader, test_loader)  # here we are using test set as validation set